[참고]
- SVM: 돌아가는 시간이 너무 오래 걸려서 중단
- Regression: LR 제외 원래 회귀 모델은 accuracy 출력이 불가하다고 함(그리고 일부 확인했을 때 매우 낮은 성능) 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

In [3]:
path = '/content/drive/MyDrive/dataset/(original) linux_memory_wo_ts.csv'
dataset = pd.read_csv(path)

In [4]:
dataset

,PID,MINFLT,MAJFLT,VSTEXT,VSIZE,RSIZE,VGROW,RGROW,MEM,label,type
0,1494,0,0,2219,390.0,82020.0,0.0,0.0,0.02,1,dos
1,1641,0,0,12,9480.0,3496.0,0.0,0.0,0.00,1,dos
2,6604,0,0,0,0.0,0.0,0.0,0.0,0.00,1,dos
3,51017,0,0,0,0.0,0.0,0.0,0.0,0.00,1,dos
4,2766,0,0,10,1.3,68724.0,0.0,0.0,0.02,1,dos
...,...,...,...,...,...,...,...,...,...,...,...
109995,4373,2076,0,148,17648.0,2408.0,0.0,0.0,0.00,0,normal
109996,1851,0,0,36,159.8,2000.0,0.0,0.0,0.00,0,normal
109997,1371,5,0,36,19296.0,640.0,0.0,0.0,0.00,0,normal
109998,1668,0,0,12,9480.0,292.0,0.0,0.0,0.00,0,normal


In [5]:
#attack_cat label encoding 후 label로 feature 명 변경
dataset['type'] = dataset['type'].replace('dos',0)
dataset['type'] = dataset['type'].replace('normal',1)

In [6]:
dataset

,PID,MINFLT,MAJFLT,VSTEXT,VSIZE,RSIZE,VGROW,RGROW,MEM,label,type
0,1494,0,0,2219,390.0,82020.0,0.0,0.0,0.02,1,0
1,1641,0,0,12,9480.0,3496.0,0.0,0.0,0.00,1,0
2,6604,0,0,0,0.0,0.0,0.0,0.0,0.00,1,0
3,51017,0,0,0,0.0,0.0,0.0,0.0,0.00,1,0
4,2766,0,0,10,1.3,68724.0,0.0,0.0,0.02,1,0
...,...,...,...,...,...,...,...,...,...,...,...
109995,4373,2076,0,148,17648.0,2408.0,0.0,0.0,0.00,0,1
109996,1851,0,0,36,159.8,2000.0,0.0,0.0,0.00,0,1
109997,1371,5,0,36,19296.0,640.0,0.0,0.0,0.00,0,1
109998,1668,0,0,12,9480.0,292.0,0.0,0.0,0.00,0,1


In [7]:
X = dataset.iloc[:,:-1]
y = dataset.iloc[:,-1]

In [8]:
X

,PID,MINFLT,MAJFLT,VSTEXT,VSIZE,RSIZE,VGROW,RGROW,MEM,label
0,1494,0,0,2219,390.0,82020.0,0.0,0.0,0.02,1
1,1641,0,0,12,9480.0,3496.0,0.0,0.0,0.00,1
2,6604,0,0,0,0.0,0.0,0.0,0.0,0.00,1
3,51017,0,0,0,0.0,0.0,0.0,0.0,0.00,1
4,2766,0,0,10,1.3,68724.0,0.0,0.0,0.02,1
...,...,...,...,...,...,...,...,...,...,...
109995,4373,2076,0,148,17648.0,2408.0,0.0,0.0,0.00,0
109996,1851,0,0,36,159.8,2000.0,0.0,0.0,0.00,0
109997,1371,5,0,36,19296.0,640.0,0.0,0.0,0.00,0
109998,1668,0,0,12,9480.0,292.0,0.0,0.0,0.00,0


In [9]:
y

0         0
1         0
2         0
3         0
4         0
         ..
109995    1
109996    1
109997    1
109998    1
109999    1
Name: type, Length: 110000, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.2, 
                                                    random_state = 0,
                                                    stratify=y)

# **Decision Tree**

In [11]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import plot_confusion_matrix # will plot the confusion matrix
import time
model_performance = pd.DataFrame(columns=['Accuracy','Recall','Precision','F1-Score','time to train','time to predict','total time'])

In [12]:
%%time
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier().fit(X_train,y_train)
y_predictions = model.predict(X_test) # These are the predictions from the test data.

CPU times: user 66.6 ms, sys: 12.1 ms, total: 78.7 ms
Wall time: 81.9 ms


In [13]:
accuracy = accuracy_score(y_test, y_predictions)
recall = recall_score(y_test, y_predictions, average='weighted')
precision = precision_score(y_test, y_predictions, average='weighted')
f1s = f1_score(y_test, y_predictions, average='weighted')

print("Accuracy: "+ "{:.2%}".format(accuracy))
print("Recall: "+ "{:.2%}".format(recall))
print("Precision: "+ "{:.2%}".format(precision))
print("F1-Score: "+ "{:.2%}".format(f1s))

Accuracy: 100.00%
Recall: 100.00%
Precision: 100.00%
F1-Score: 100.00%


# **Logistic regression**

In [14]:
# 필요한 모듈 import
from sklearn.linear_model import LogisticRegression 
import sklearn.metrics

clf = LogisticRegression().fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [15]:
result_LR = clf.predict(X_test)

In [16]:
print('Accuracy: ', sklearn.metrics.accuracy_score(y_test,result_LR))

Accuracy:  0.9654545454545455


# **Naive Bayes**

In [17]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()

In [18]:
result_nb = nb.fit(X_train, y_train).predict(X_test)

In [19]:
print('Accuracy: ', sklearn.metrics.accuracy_score(y_test,result_nb))

Accuracy:  0.11768181818181818


# **Lasso**

In [20]:
from sklearn.linear_model import Lasso

lasso = Lasso().fit(X_train, y_train)

In [21]:
print("테스트 세트의 정확도 : {:.2f}".format(lasso.score(X_test, y_test)))

테스트 세트의 정확도 : 0.03


# **KNN**

In [22]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [23]:
result_knn=knn.predict(X_test)

In [24]:
print('Accuracy: ', sklearn.metrics.accuracy_score(y_test,result_knn))

Accuracy:  0.9957727272727273


# **Random forest**

In [25]:
# Random forest
from sklearn.ensemble import RandomForestClassifier
# instantiate the classifier 
RF = RandomForestClassifier(random_state=0)
RF.fit(X_train, y_train)
rf_pred = RF.predict(X_test)

In [26]:
from sklearn.metrics import accuracy_score
accuracy_rf = accuracy_score(y_test, rf_pred)
print("Accuracy: {}".format(accuracy_rf))

Accuracy: 1.0


# **MLP**

In [31]:
import keras
from keras.layers import Dense
from keras.models import Sequential

model = Sequential()
model.add(Dense(7,input_shape=(X.shape[1],),activation='relu'))
model.add(Dense(4,activation='relu'))
model.add(Dense(4,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [32]:
model.compile(optimizer='adam', metrics=['accuracy'], loss='binary_crossentropy')
a = model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test),shuffle=True)

Epoch 1/5
2750/2750 [==============================] - 9s 3ms/step - loss: 4.9955 - accuracy: 0.8679 - val_loss: 0.6567 - val_accuracy: 0.9091
Epoch 2/5
2750/2750 [==============================] - 9s 3ms/step - loss: 0.5102 - accuracy: 0.8919 - val_loss: 0.3628 - val_accuracy: 0.9109
Epoch 3/5
2750/2750 [==============================] - 8s 3ms/step - loss: 0.3020 - accuracy: 0.9059 - val_loss: 0.2969 - val_accuracy: 0.9062
Epoch 4/5
2750/2750 [==============================] - 10s 4ms/step - loss: 0.3024 - accuracy: 0.9081 - val_loss: 0.2958 - val_accuracy: 0.9107
Epoch 5/5
2750/2750 [==============================] - 9s 3ms/step - loss: 0.3032 - accuracy: 0.9098 - val_loss: 0.2950 - val_accuracy: 0.9107


In [33]:
model.compile(optimizer='adam', metrics=['accuracy'], loss='binary_crossentropy')
a = model.fit(X_train,y_train,epochs=5,shuffle=True)

Epoch 1/5
2750/2750 [==============================] - 6s 2ms/step - loss: 0.3005 - accuracy: 0.9090
Epoch 2/5
2750/2750 [==============================] - 5s 2ms/step - loss: 0.2997 - accuracy: 0.9091
Epoch 3/5
2750/2750 [==============================] - 5s 2ms/step - loss: 0.2985 - accuracy: 0.9091
Epoch 4/5
2750/2750 [==============================] - 5s 2ms/step - loss: 0.2955 - accuracy: 0.9091
Epoch 5/5
2750/2750 [==============================] - 5s 2ms/step - loss: 0.2969 - accuracy: 0.9091


# **SVM (시간이 너무 길게 소요돼서 중단)**

In [ ]:
from sklearn.svm import SVC

SVM = SVC(kernel='rbf')
SVM.fit(X_train, y_train)

In [ ]:
y_pred = SVM.predict(X_test)

In [ ]:
from sklearn import metrics

# Model Accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# **Linear regression-accuracy 산출 불가**

In [ ]:
from sklearn.linear_model import LinearRegression

linr = LinearRegression()
result_linr = linr.fit(X_train, y_train)

In [ ]:
result_linr

LinearRegression()

In [ ]:
pred = linr.predict(X_test)
accuracy_score(X_test,pred)

ValueError: ignored

In [ ]:
result_linr = linr.fit(X_train, y_train).predict(X_test)

In [ ]:
result_linr

array([-4.57411886e-14,  1.00000000e+00,  1.00000000e+00, ...,
        1.00000000e+00,  1.00000000e+00,  1.00000000e+00])

In [ ]:
print('Accuracy: ', sklearn.metrics.accuracy_score(y_test,result_linr))

ValueError: ignored